In [ ]:
import ipyparallel
import socket
import os
import getpass

import numpy as np
import tools
import funcs
from operator import itemgetter
import adaptive
adaptive.notebook_extension()
import systems
import matplotlib.pyplot as plt
import kwant

In [ ]:
client = ipyparallel.Client(
    profile='slurm2104', 
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client[:].use_cloudpickle()

ids = client.ids
hosts = client[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
client2 = ipyparallel.Client(
    profile='slurm2102', 
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client2[:].use_cloudpickle()

ids = client2.ids
hosts = client2[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client2[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
def reload():
    import importlib
    import funcs
    import tools
    import systems
    
    importlib.reload(funcs)
    importlib.reload(tools)
    importlib.reload(systems)
reload()

In [ ]:
np.random.default_rng(0).uniform(-.05,.05,None)

## First runner

In [ ]:
_ = client[:].apply_sync(reload)

In [ ]:
m0  = [-0.017, 0.006]
m1  = [ 16.35, 13.83]
D   = [-13.31,-13.38]
v_F = [ 1.920, 2.887]
m_zs =[  0.02, 0.02]
mus = [0.0135, 0.001]
Smag_imp = [0.01,0.01]

j = 0

qa = tools.QuickAdptive(
    funcs.finite_spectrum,
    fname_prefix='data_learner_',
    learner_bounds_names=['Smag_imp'],
    arg_picker=itemgetter('Es_mean'),
    a=[10],
    a_z=[10],
    L=[30000],
    W=[500],
    T=[1],
    delta=[1e-3],
    mu=[mus[j]-(m0[j]+m_zs[j])*D[j]/m1[j]],
    Smag_imp=[0,0.02],
    m_z=[m_zs[j]],
    correlation_length=[20],
    n_av=[20],
    k=[30],
    sum=[True],
    m0=[m0[j]],
    m1=[m1[j]],
    D=[D[j]],
    v_F=[v_F[j]]
)

In [ ]:
qa = tools.QuickAdptive(
    funcs.finite_spectrum_deltamu,
    fname_prefix='data_learner_',
    learner_bounds_names=['Smag_imp'],
    arg_picker=itemgetter('Es_mean'),
    a=[10],
    a_z=[10],
    L=[30000],
    W=[500],
    T=[1],
    delta=[1e-3],
    mu=[-(m0[j]+m_zs[j])*D[j]/m1[j]],
    deltamu=[.05],
    Smag_imp=[0,0.02],
    m_z=[m_zs[j]],
    correlation_length=[20],
    n_av=[20],
    k=[30],
    sum=[True],
    m0=[m0[j]],
    m1=[m1[j]],
    D=[D[j]],
    v_F=[v_F[j]]
)

In [ ]:
executor = client.executor()
job = executor.submit(funcs.finite_spectrum, a=4, a_z=4, L=1000, T=1, W=500, delta=0.001,mu=0.01, Smag_imp=0.01, m_z=0, correlation_length=20, n_av=1, k=30, sum=True, m0=-0.017, m1=16.35, D=-13.31, v_F=1.920)

In [ ]:
res = job.result()

In [ ]:
qa.load('./data/finite_lenght_spectrum')

In [ ]:
import adaptive
adaptive.notebook_extension()

qa.runn(
    goal=lambda learner: learner.npoints > 13,
    executor=client,
    periodic_saving_folder='./data/finite_lenght_spectrum',
    interval=60*60
)

In [ ]:
qa.runner.live_info()

In [ ]:
qa.save('./data/finite_lenght_spectrum')

In [ ]:
Es_k0 = np.array([-0.07037036, -0.06908563, -0.06347416, -0.06312472, -0.05727571,
       -0.0559619 , -0.05043639, -0.04934656, -0.04397647, -0.04289999,
       -0.04057224, -0.03460804, -0.02549919, -0.01550431, -0.0019264 ,
        0.00679332,  0.01989821,  0.03389364,  0.03722355,  0.04465   ,
        0.04897276,  0.05569907,  0.06518542,  0.06941859,  0.0825615 ])

In [ ]:
def even_or_odd(mu_test):
    nu_bands = (np.sum(Es_k0[Es_k0<0] > mu_test)+np.sum(Es_k0[Es_k0>0] < mu_test))
    if nu_bands != 0:
        return nu_bands%2
    else:
        return -1

In [ ]:
mus_run = np.array([np.random.default_rng(i).uniform(-.05,.05,None) for i in range(200)])

In [ ]:
mus_run_parity = np.array([even_or_odd(mu) for mu in mus_run])

In [ ]:
even = np.bool8(mus_run_parity) & (mus_run_parity!=-1)
odd = ~np.bool8(mus_run_parity) & (mus_run_parity!=-1)
no_states = mus_run_parity==-1

In [ ]:
for e in Es_k0:
    plt.plot([-.01,.01],[e]*2,"k-")
plt.plot(np.zeros(len(mus_run[even])), mus_run[even], "rx")
plt.plot(np.zeros(len(mus_run[odd])), mus_run[odd], "bx")
plt.plot(np.zeros(len(mus_run[no_states])), mus_run[no_states], "kx")

plt.ylim(-.05,.05)

In [ ]:
Data = qa.runner.learner.to_dataframe()['extra_data']
data_to_plot = np.array([Data[i]['Es_mean'] for i in range(len(Data))])
try: 
    qa.combos['Smag_imp']
    xs = np.array(qa.runner.learner.to_dataframe()['x'])/1e4
except:
    xs = np.array(qa.runner.learner.to_dataframe()['x'])*1e3
cutoff = .5e-3
middle = 15
std = []
for i in range(len(Data)):
    std.append( np.std(np.array([Data[i]['Es'][ii] for ii in range(20)]),axis=0))
std = np.array(std)
for i in range(len(std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    plt.fill(np.append(xs,xs[::-1]),np.append(data_to_plot[:,i]-std[:,i],(data_to_plot[:,i]+std[:,i])[::-1]),color=c,alpha=.2)   

plt.plot(xs,data_to_plot[:,:middle-1],"k-");
plt.plot(xs,data_to_plot[:,middle-1:middle+1],"r-");
plt.plot(xs,data_to_plot[:,middle+1:],"k-");
#plt.plot(xs[i],data_to_plot[np.where(data_to_plot>0)[0][0]],"r_",ms=10)
#plt.plot(xs[i],data_to_plot[np.where(data_to_plot<0)[0][-1]],"r_",ms=10)
plt.ylim(-cutoff,cutoff)
plt.yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
#

plt.ylabel("$E~[meV]$");
try: 
    qa.combos['Smag_imp']
    plt.ylim(-1e-3,1e-3);
    plt.xlim(.5,4);
    plt.xlabel("$L~[nm]$");
    plt.savefig("L_"+["NI","QSHI"][j]+".png")
except:
    plt.ylim(-.5e-3,.5e-3);
    plt.xlim(0,20);
    plt.xlabel("$\delta \mu~[meV]$");
    plt.savefig("Smag_imp_"+["NI","QSHI"][j]+".png")

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5),sharey=True,sharex=True,layout='constrained')

Data = qa.runner.learner.to_dataframe()['extra_data']
data_to_plot = np.array([Data[i]['Es_mean'] for i in range(len(Data))])
try: 
    qa.combos['Smag_imp']
    xs = np.array(qa.runner.learner.to_dataframe()['x'])/1e4
except:
    xs = np.array(qa.runner.learner.to_dataframe()['x'])*1e3
cutoff = .5e-3
std = []
odd_std,even_std,no_states_std = [],[],[]
odd_Es,even_Es,no_states_Es = [],[],[]
middle = 15

for i in range(len(Data)):
    std.append( np.std(np.array([Data[i]['Es'][ii] for ii in range(20)]),axis=0))
    odd_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[odd]]),axis=0))
    even_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[even]]),axis=0))
    no_states_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[no_states]]),axis=0))
    odd_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[odd]]))
    even_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[even]]))
    no_states_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[no_states]]))

odd_std = np.array(odd_std)
even_std = np.array(even_std)
no_states_std = np.array(no_states_std)
odd_Es = np.mean(np.array(odd_Es),axis=1)
even_Es = np.mean(np.array(even_Es),axis=1)
no_states_Es = np.mean(np.array(no_states_Es),axis=1)
for i in range(len(odd_std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    ax[0].fill(np.append(xs,xs[::-1]),np.append(odd_Es[:,i]-odd_std[:,i],(odd_Es[:,i]+odd_std[:,i])[::-1]),color=c,alpha=.2)   

ax[0].plot(xs,odd_Es[:,:middle-1],"k-");
ax[0].plot(xs,odd_Es[:,middle-1:middle+1],"r-");
ax[0].plot(xs,odd_Es[:,middle+1:],"k-");

for i in range(len(even_std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    ax[1].fill(np.append(xs,xs[::-1]),np.append(even_Es[:,i]-even_std[:,i],(even_Es[:,i]+even_std[:,i])[::-1]),color=c,alpha=.2)   

ax[1].plot(xs,even_Es[:,:middle-1],"k-");
ax[1].plot(xs,even_Es[:,middle-1:middle+1],"r-");
ax[1].plot(xs,even_Es[:,middle+1:],"k-");

ax[0].set_ylim(-cutoff,cutoff)
ax[0].set_yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
ax[1].set_yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
ax[0].set_xlim(0,20);

ax[0].set_xlabel("$\delta \mu~[meV]$"); ax[1].set_xlabel("$\delta \mu~[meV]$");
ax[0].set_ylabel("$E~[meV]$"); 
ax[0].set_ylim(-.5e-3,.5e-3);

ax[0].set_title("Odd nu. of bands"); ax[1].set_title("Even nu. of bands");
try: 
    qa.combos['Smag_imp']
    ax[0].set_xlim(.5,4);
    ax[0].set_xlabel("$L~[nm]$"); ax[1].set_xlabel("$L~[nm]$");
    plt.savefig("L_mu_"+["NI","QSHI"][j]+"2.png")
except:
    ax[0].plot([[10,3][jj]]*2,[-1,1],"k:",zorder=-1)
    ax[0].set_xlim(0,20);
    ax[0].set_xlabel("$\delta \mu~[meV]$"); ax[1].set_xlabel("$\delta \mu~[meV]$");
    plt.savefig("Smag_imp_mu_"+["NI","QSHI"][j]+"2.png")

## Second runner

In [ ]:
_ = client2[:].apply_sync(reload)

In [ ]:
jj = 0

qa2 = tools.QuickAdptive(
    funcs.finite_spectrum,
    fname_prefix='data_learner_',
    learner_bounds_names=['L'],
    arg_picker=itemgetter('Es_mean'),
    a=[10],
    a_z=[10],
    L=[5000,40000],
    W=[500],
    T=[1],
    delta=[1e-3],
    mu=[mus[jj]-(m0[jj]+m_zs[jj])*D[jj]/m1[jj]],
    Smag_imp=[0.01],
    m_z=[m_zs[jj]],
    correlation_length=[20],
    n_av=[20],
    k=[30],
    sum=[True],
    m0=[m0[jj]],
    m1=[m1[jj]],
    D=[D[jj]],
    v_F=[v_F[jj]]
)

In [ ]:
qa2 = tools.QuickAdptive(
    funcs.finite_spectrum_deltamu,
    fname_prefix='data_learner_',
    learner_bounds_names=['L'],
    arg_picker=itemgetter('Es_mean'),
    a=[10],
    a_z=[10],
    L=[5000, 40000],
    W=[500],
    T=[1],
    delta=[1e-3],
    mu=[-(m0[jj]+m_zs[jj])*D[jj]/m1[jj]],
    deltamu=[.05],
    Smag_imp=[0.01],
    m_z=[m_zs[jj]],
    correlation_length=[20],
    n_av=[20],
    k=[30],
    sum=[True],
    m0=[m0[jj]],
    m1=[m1[jj]],
    D=[D[jj]],
    v_F=[v_F[jj]]
)

In [ ]:
qa2.load('./data/finite_lenght_spectrum')

In [ ]:
import adaptive
adaptive.notebook_extension()

qa2.runn(
    goal=lambda learner: learner.npoints > 13,
    executor=client2,
    periodic_saving_folder='./data/finite_lenght_spectrum',
    interval=60*60
)

In [ ]:
qa2.runner.live_info()

In [ ]:
qa2.save('./data/finite_lenght_spectrum')

In [ ]:
Data = qa2.runner.learner.to_dataframe()['extra_data']
data_to_plot = np.array([Data[i]['Es_mean'] for i in range(len(Data))])
try: 
    qa2.combos['Smag_imp']
    xs = np.array(qa2.runner.learner.to_dataframe()['x'])/1e4
except:
    xs = np.array(qa2.runner.learner.to_dataframe()['x'])*1e3
cutoff = .5e-3
std = []
middle = 15

for i in range(len(Data)):
    std.append( np.std(np.array([Data[i]['Es'][ii] for ii in range(20)]),axis=0))
std = np.array(std)
for i in range(len(std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    plt.fill(np.append(xs,xs[::-1]),np.append(data_to_plot[:,i]-std[:,i],(data_to_plot[:,i]+std[:,i])[::-1]),color=c,alpha=.2)   

plt.plot(xs,data_to_plot[:,:middle-1],"k-");
plt.plot(xs,data_to_plot[:,middle-1:middle+1],"r-");
plt.plot(xs,data_to_plot[:,middle+1:],"k-");
#plt.plot(xs[i],data_to_plot[np.where(data_to_plot>0)[0][0]],"r_",ms=10)
#plt.plot(xs[i],data_to_plot[np.where(data_to_plot<0)[0][-1]],"r_",ms=10)
plt.ylim(-cutoff,cutoff)
plt.yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
plt.xlim(0,20);

plt.xlabel("$\delta \mu~[meV]$");
plt.ylabel("$E~[meV]$");
plt.ylim(-.5e-3,.5e-3);
try: 
    qa2.combos['Smag_imp']
    plt.xlim(.5,4);
    plt.xlabel("$L~[nm]$");
    plt.savefig("L_"+["NI","QSHI"][jj]+"2.png")
except:
    plt.plot([[10,3][jj]]*2,[-1,1],"k:",zorder=-1)
    plt.xlim(0,20);
    plt.xlabel("$\delta \mu~[meV]$");
    plt.savefig("Smag_imp_"+["NI","QSHI"][jj]+"2.png")

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5),sharey=True,sharex=True,layout='constrained')

Data = qa2.runner.learner.to_dataframe()['extra_data']
data_to_plot = np.array([Data[i]['Es_mean'] for i in range(len(Data))])
try: 
    qa2.combos['Smag_imp']
    xs = np.array(qa2.runner.learner.to_dataframe()['x'])/1e4
except:
    xs = np.array(qa2.runner.learner.to_dataframe()['x'])*1e3
cutoff = .5e-3
std = []
odd_std,even_std,no_states_std = [],[],[]
odd_Es,even_Es,no_states_Es = [],[],[]
middle = 15

for i in range(len(Data)):
    std.append( np.std(np.array([Data[i]['Es'][ii] for ii in range(20)]),axis=0))
    odd_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[odd]]),axis=0))
    even_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[even]]),axis=0))
    no_states_std.append( np.std(np.array([Data[i]['Es'][ii] for ii in np.arange(20)[no_states]]),axis=0))
    odd_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[odd]]))
    even_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[even]]))
    no_states_Es.append( np.array([Data[i]['Es'][ii] for ii in np.arange(20)[no_states]]))

odd_std = np.array(odd_std)
even_std = np.array(even_std)
no_states_std = np.array(no_states_std)
odd_Es = np.mean(np.array(odd_Es),axis=1)
even_Es = np.mean(np.array(even_Es),axis=1)
no_states_Es = np.mean(np.array(no_states_Es),axis=1)
for i in range(len(odd_std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    ax[0].fill(np.append(xs,xs[::-1]),np.append(odd_Es[:,i]-odd_std[:,i],(odd_Es[:,i]+odd_std[:,i])[::-1]),color=c,alpha=.2)   

ax[0].plot(xs,odd_Es[:,:middle-1],"k-");
ax[0].plot(xs,odd_Es[:,middle-1:middle+1],"r-");
ax[0].plot(xs,odd_Es[:,middle+1:],"k-");

for i in range(len(even_std[0,:])):
    if i == middle-1 or i == middle:
        c = "r"
    else:
        c = "k"
    ax[1].fill(np.append(xs,xs[::-1]),np.append(even_Es[:,i]-even_std[:,i],(even_Es[:,i]+even_std[:,i])[::-1]),color=c,alpha=.2)   

ax[1].plot(xs,even_Es[:,:middle-1],"k-");
ax[1].plot(xs,even_Es[:,middle-1:middle+1],"r-");
ax[1].plot(xs,even_Es[:,middle+1:],"k-");

ax[0].set_ylim(-cutoff,cutoff)
ax[0].set_yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
ax[1].set_yticks(np.arange(-1e-3,1.1e-3,.2e-3),[np.round(i,1) for i in np.arange(-1,1.1,.2)]);
ax[0].set_xlim(0,20);

ax[0].set_xlabel("$\delta \mu~[meV]$"); ax[1].set_xlabel("$\delta \mu~[meV]$");
ax[0].set_ylabel("$E~[meV]$"); 
ax[0].set_ylim(-.5e-3,.5e-3);

ax[0].set_title("Odd nu. of bands"); ax[1].set_title("Even nu. of bands");
try: 
    qa2.combos['Smag_imp']
    ax[0].set_xlim(.5,4);
    ax[0].set_xlabel("$L~[nm]$"); ax[1].set_xlabel("$L~[nm]$");
    plt.savefig("L_mu_"+["NI","QSHI"][jj]+"2.png")
except:
    ax[0].set_plot([[10,3][jj]]*2,[-1,1],"k:",zorder=-1)
    ax[0].set_xlim(0,20);
    ax[0].set_xlabel("$\delta \mu~[meV]$"); ax[1].set_xlabel("$\delta \mu~[meV]$");
    plt.savefig("Smag_imp_mu_"+["NI","QSHI"][jj]+"2.png")

In [ ]:
odd_std.shape, np.mean(odd_Es,axis=1).shape

In [ ]:
qa2.runner.task.result()

In [ ]:
fsyst = systems.make_ti_ribbon(10,10, 500, 500, 1)

In [ ]:
S_imp = funcs.get_S_imp(0, 10, fsyst, 20)

In [ ]:
disorder_profile = np.zeros((51,51))
for i in range(51):
    for j in range(51):
        disorder_profile[i,j] = S_imp(fsyst.sites[i*51+j],1)

In [ ]:
import matplotlib.pyplot as plt
import pickle

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
im = ax.contourf(disorder_profile)
plt.colorbar(im)

In [ ]:
fileName = 'disorder_profile.pkl'
fileObject = open(fileName, 'wb')

with open(fileName,'wb') as f:
    pickle.dump(disorder_profile, f) 

In [ ]:
plt.hist(disorder_profile.flatten())